# pix2pix
### Rešenje problema transformacije slike u sliku

Problem transformacije slike u sliku je prisutan u mnogobrojnim problemima u procesiranju slika, računarskoj grafici i računarskom vidu (computer vision). Predstavlja transformaciju gde se slika prevodi iz jedne reprezentacije u drugu.

Za izradu pix2pix modela korišćen je cGAN - conditional Generative Adversial Networks. 

Za razliku od klasičnih GAN-ova:
- cGAN generiše slike na osnovu ulazne slike koja nije "white-noise image" i predstavlja početnu reprezentaciju slike
- cGAN diskriminator upoređuje parove slika - par (generisana slika, ciljna slika) i (ulazna slika, ciljna slika)

![Primeri parova iz skupova podataka korišćenih u originalnom radu](assets/image_pairs.png)

### Funkcija greške
$$G∗ = \argmin_{G}\max_{D} L_{cGAN} (G, D) + λL_{L1}(G)$$

gde je $L_{cGAN}$:
$$ L_{cGAN}(G, D) = E_{x,y} [log D(x, y)]+ E_{x,z} [log(1 − D(x, G(x, z))]$$

(odnosno računa se greška u odnosu na ulaznu sliku )

Parametar $\lambda$ može imati vrednosti iz opsega [0, 100]. Za $\lambda = 0$, funkcija greške računa samo grešku $L_{cGAN}$ i takav model proizvodi oštrije slike, ali unosi vizuelne defekte. Za $\lambda = 1000$, smanjuju se vizuelni defekti.

Kada bi se greška zasnivala samo na L1 normi, model bi generisao mutne slike. 

![title](assets/lambda.png)

## Arhitektura modela
### Diskriminator
Diskriminator se sastoji od 5 slojeva:
1. Conv2D, LeakyRelu
2. Conv2D, BatchNorm, LeakyRelu
3. Conv2D, BatchNorm, LeakyRelu
4. Conv2D, BatchNorm, LeakyRelu
5. Conv2D

![discr](assets/discr_architecture.png)

### Generator
Generator ima enkoder-dekoder arhitekturu sa skip konekcijama (po ugledu na UNet).
#### Enkoder
1. Conv2D, LeakyRelu
2. Conv2D, BatchNorm, LeakyRelu
3. Conv2D, BatchNorm, LeakyRelu
4. Conv2D, BatchNorm, LeakyRelu

#### Dekoder
1. ConvTranspose2d, BatchNorm2d, ReLU, Dropout
2. ConvTranspose2d, BatchNorm2d, ReLU, Dropout
3. ConvTranspose2d, BatchNorm2d, ReLU, Dropout
4. ConvTranspose2d, BatchNorm2d, ReLU
5. ConvTranspose2d, BatchNorm2d, ReLU
6. ConvTranspose2d, BatchNorm2d, ReLU
7. ConvTranspose2d, BatchNorm2d, ReLU
8. ReLU, ConvTranspose2d, Tanh

![gen](assets/gen_architecture.png)

### Skip konekcije
Problem u neuronskim mrežama jeste gubitak informacije kada podatak prolazi kroz mnogo slojeva. Ovo otežava učenje sitnih detalja. Skip konekcije direktno prosleđuju informacije od jednog sloja enkodera ka jednom sloju dekodera, zaobilazeći slojeve između njih. Na ovaj način se čuvaju informacije o detaljima. Eksperimentalno je dokazano da skip konekcije pomažu konvergenciji modela jer ublažuju problem nedostajućih gradijenata.
Skip konekcije u pix2pix modelu povezuju sloj $i$ enkodera sa $n-i$ slojem dekodera i ostvaruje se konkatenacijom tenzora.

## Treniranje modela

Treniranje modela je izvršeno u GoogleColab-u. Koriščen je T4 GPU. Treniranje je u proseku trajalo 2h za 400 slika iz facades skupa.

Izazovi:
- kreiranje skip konekcija (određivanje veličinu ulaza/izlaza sloja)
- ograničeni resursi, učitavanje podataka bi pojelo dosta CPU resursa (sređeno povećanjem broja worker-a u dataloader-u)
- ni jedan drugi skup nije iskorišćen zbog ograničenih resursa